In [ ]:
!apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null  
!wget -q https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!tar xf spark-3.3.1-bin-hadoop3.tgz  
!pip install -q findspark
!pip install isbnlib

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://security.ubuntu.com/ubuntu bionic-sec

In [ ]:
import findspark
findspark.init("spark-3.3.1-bin-hadoop3")
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
from pyspark.sql import Row
import pyspark.sql.functions as f
from pyspark.sql.functions import format_number
from pyspark.sql.functions import col, split, when
import pandas as pd
spark = SparkSession.builder.appName("Basic").getOrCreate()
import requests as req
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import ltrim
from pyspark.sql.functions import *
from functools import reduce
from pyspark.sql import DataFrame
from isbnlib import is_isbn10, is_isbn13
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
import re
from pyspark.sql.functions import col,isnan,when,count
from pyspark.sql.functions import row_number, monotonically_increasing_id
from pyspark.sql import Window
from pyspark.sql.functions import trim
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
Books = spark.read.option("header", True).csv('/content/drive/MyDrive/Books_Rating/DataBase/BX-Books.csv')
Books.show(truncate = False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|"ISBN";"Book-Title";"Book-Author";"Year-Of-Publication";"Publisher";"Image-URL-S";"Image-URL-M";"Image-URL-L"                                                                                                                                                                                                                                                  |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
Books = Books.withColumnRenamed('"ISBN";"Book-Title";"Book-Author";"Year-Of-Publication";"Publisher";"Image-URL-S";"Image-URL-M";"Image-URL-L"', "xx")
Books = Books.withColumn('xx', regexp_replace('xx', '"', ''))

In [ ]:
Books = Books.withColumn('BookNumber', split(Books['xx'], ';').getItem(0)) \
       .withColumn('BookName', split(Books['xx'], ';').getItem(1)) \
       .withColumn('Author', split(Books['xx'], ';').getItem(2)) \
       .withColumn('Year', split(Books['xx'], ';').getItem(3)) \
       .withColumn('Publisher', split(Books['xx'], ';').getItem(4)) \
       .withColumn('one', split(Books['xx'], ';').getItem(5)) \
       .withColumn('two', split(Books['xx'], ';').getItem(6)) \
       .withColumn('three', split(Books['xx'], ';').getItem(7)) \
       .withColumn('four', split(Books['xx'], ';').getItem(8)).drop("xx")
Books = Books.na.fill({'Author': 'unknown','Year': 0, 'Publisher': 'unknown'})

In [ ]:
Books.where("Publisher like '%Penny Frank%' ").show()

+----------+--------+------+----+---------+---+---+-----+----+
|BookNumber|BookName|Author|Year|Publisher|one|two|three|four|
+----------+--------+------+----+---------+---+---+-----+----+
+----------+--------+------+----+---------+---+---+-----+----+



In [ ]:
t = Books.where("rlike(Year,'^[a-zA-Z]*$')")
u = Books.where("Year BETWEEN 0 AND 2050")
v = Books.where("Publisher BETWEEN 0 AND 2050")
w = Books.where("one BETWEEN 0 AND 2050")
x = Books.where("two BETWEEN 0 AND 2050")
y = Books.where("three BETWEEN 0 AND 2050")
z = Books.where("four BETWEEN 0 AND 2050")
xx = t.where("Publisher BETWEEN 0 AND 2050")
zz = t.where("one BETWEEN 0 AND 2050")

In [ ]:
u = u.drop("Publisher", "one", "two", "three", "four").withColumn("Year",col("Year").cast("int"))

v = v.withColumnRenamed("Author", "BookName2").withColumnRenamed("Year", "Author").withColumnRenamed("Publisher", "Year").withColumnRenamed("one", "Publisher") \
    .withColumn("BookName",concat(col("BookName"),lit(' -'),col("BookName2"))).drop("BookName2","Publisher", "two", "three", "four").withColumn("Year",col("Year").cast("int"))

w = w.withColumnRenamed("Author", "BookName2").withColumnRenamed("Year", "Author").withColumnRenamed("Publisher", "Author2").withColumnRenamed("one", "Year") \
    .withColumnRenamed("two", "Publisher").withColumn("BookName",concat(col("BookName"),lit(' -'),col("BookName2"))).withColumn("Author",concat(col("Author"),lit(' -'),col("Author2"))) \
    .drop("BookName2","Author2","Publisher", "three", "four").withColumn("Year",col("Year").cast("int"))

x = x.withColumnRenamed("Author", "BookName2").withColumnRenamed("Year", "Author").withColumnRenamed("Publisher", "Author2").withColumnRenamed("one", "Author3").withColumnRenamed("two", "Year") \
    .withColumnRenamed("three", "Publisher").withColumn("BookName",concat(col("BookName"),lit(' -'),col("BookName2"))).withColumn("Author",concat(col("Author"),lit(' -'),col("Author2"))) \
    .withColumn("Author",concat(col("Author"),lit(' -'),col("Author3"))).drop("BookName2","Author2","Author3","Publisher","four").withColumn("Year",col("Year").cast("int"))

y = y.withColumnRenamed("Author", "BookName2").withColumnRenamed("Year", "BookName3").withColumnRenamed("Publisher", "Author").withColumnRenamed("one", "Author2").withColumnRenamed("two", "Author3") \
    .withColumnRenamed("three", "Year").withColumnRenamed("four", "Publisher").withColumn("BookName",concat(col("BookName"),lit(' -'),col("BookName2"))) \
    .withColumn("BookName",concat(col("BookName"),lit(' -'),col("BookName3"))).withColumn("Author",concat(col("Author"),lit(' -'),col("Author2"))) \
    .withColumn("Author",concat(col("Author"),lit(' -'),col("Author3"))).drop("BookName2","BookName3","Author2","Author3","Publisher").withColumn("Year",col("Year").cast("int"))

z = z.withColumnRenamed("Author", "BookName2").withColumnRenamed("Year", "BookName3").withColumnRenamed("Publisher", "Author").withColumnRenamed("one", "Author2").withColumnRenamed("two", "Author3") \
    .withColumnRenamed("three", "Author4").withColumnRenamed("four", "Year").withColumn("BookName",concat(col("BookName"),lit(' -'),col("BookName2"))) \
    .withColumn("BookName",concat(col("BookName"),lit(' -'),col("BookName3"))).withColumn("Author",concat(col("Author"),lit(' -'),col("Author2"))).withColumn("Author",concat(col("Author"),lit(' -'),col("Author3"))) \
    .withColumn("Author",concat(col("Author"),lit(' -'),col("Author4"))).drop("BookName2","BookName3","Author2","Author3","Author4").withColumn("Year",col("Year").cast("int"))

xx = xx.withColumnRenamed("Author", "BookName2").withColumnRenamed("Year", "Author").withColumnRenamed("Publisher", "Year").withColumnRenamed("one", "Publisher") \
    .withColumn("BookName",concat(col("BookName"),lit(' -'),col("BookName2"))).drop("BookName2","Publisher", "two", "three", "four").withColumn("Year",col("Year").cast("int"))

zz = zz.withColumnRenamed("Author", "BookName2").withColumnRenamed("Year", "Author").withColumnRenamed("Publisher", "Author2").withColumnRenamed("one", "Year") \
    .withColumnRenamed("two", "Publisher").withColumn("BookName",concat(col("BookName"),lit(' -'),col("BookName2"))).withColumn("Author",concat(col("Author"),lit(' -'),col("Author2"))) \
    .drop("BookName2","Author2","Publisher", "three", "four").withColumn("Year",col("Year").cast("int"))               

In [ ]:
df = [u,v,w,x,y,z,xx,zz]
Books = reduce(DataFrame.unionAll, df)

In [ ]:
Books = Books.na.fill(value=0,subset=["Year"]).withColumn("BookNumber",ltrim(col("BookNumber"))).withColumn("BookName",ltrim(col("BookName"))) \
        .withColumn("Author",ltrim(col("Author"))).withColumn("Year",ltrim(col("Year"))).withColumn('BookName', regexp_replace('BookName', "$", '')) \
        .withColumn('BookName', regexp_replace('BookName', "#", '')).withColumn('BookName', regexp_replace('BookName', "�", '')) \
        .withColumn('BookName', regexp_replace('BookName', "-", '')).withColumn('BookName', regexp_replace('BookName', "¿", '')) \
        .withColumn('BookName', regexp_replace('BookName', "°", '')).withColumn('BookName', regexp_replace('BookName', "¶", '')) \
        .withColumn('BookName', regexp_replace('BookName', '/', '')).withColumn('BookName', regexp_replace('BookName', "\*", '')) \
        .withColumn('BookName', regexp_replace('BookName', "\+", ''))
Books = Books.select([f.col(col).alias(re.sub("-","_",col)) for col in Books.columns])
Books = Books.select([f.col(col).alias(re.sub("[^0-9a-zA-Z_$]+","",col)) for col in Books.columns])

In [ ]:
ValidISBN = []
InvalidISBN = []

row_listBOOKS = Books.collect()

for i in row_listBOOKS:
  if (is_isbn10(i.__getitem__('BookNumber')) == True):
    ValidISBN.append(i)
  else:
    InvalidISBN.append(i) 

InvalidISBN_ONLY = []
for i in InvalidISBN:
  InvalidISBN_ONLY.append(i.__getitem__('BookNumber')) 

Books = Books.filter(~col('BookNumber').isin(InvalidISBN_ONLY))

In [ ]:
Books = Books.withColumnRenamed('BookNumber', 'BookBK')
Books = Books.withColumn("Year", when(Books.Year < 1000 ,0).when(Books.Year > 2022 ,0).otherwise(Books.Year))
Books = Books.withColumn("BookSK",row_number().over(Window.orderBy(monotonically_increasing_id()))+100)
DIM_BOOKS = Books.select('BookSK', 'BookBK', 'BookName', 'Author', 'Year')


DIM_BOOKS.show(truncate = False)
DIM_BOOKS.describe().show()

+------+----------+--------------------------------------------------------------------------------------------------+--------------------+----+
|BookSK|BookBK    |BookName                                                                                          |Author              |Year|
+------+----------+--------------------------------------------------------------------------------------------------+--------------------+----+
|101   |0195153448|Classical Mythology                                                                               |Mark P. O. Morford  |2002|
|102   |0002005018|Clara Callan                                                                                      |Richard Bruce Wright|2001|
|103   |0060973129|Decision in Normandy                                                                              |Carlo D'Este        |1991|
|104   |0374157065|Flu: The Story of the Great Influenza Pandemic of 1918 and the Search for the Virus That Caused It|Gina Bari Ko

In [ ]:
DIM_BOOKS.createTempView("Books")

spark.sql(
    '''
    select * from Books
    where Year not BETWEEN 1000 AND 2022 and Year !=0
    '''
).show()

+------+------+--------+------+----+
|BookSK|BookBK|BookName|Author|Year|
+------+------+--------+------+----+
+------+------+--------+------+----+



In [ ]:
spark.sql(
    '''
    select * from Books
    where length(BookBK) != 10
    '''
).show()

+------+------------+--------------------+-------------+----+
|BookSK|      BookBK|            BookName|       Author|Year|
+------+------------+--------------------+-------------+----+
|110325|0486404242\t|War in Kind: And ...|Stephen Crane|1998|
+------+------------+--------------------+-------------+----+



In [ ]:
DIM_BOOKS = spark.sql(
    '''
    select * from Books
    where length(BookBK) = 10
    '''
)

In [ ]:
DIM_BOOKS.createTempView("xxx")

spark.sql(
    '''
    select * from xxx
    where length(BookBK) != 10
    '''
).show()

+------+------+--------+------+----+
|BookSK|BookBK|BookName|Author|Year|
+------+------+--------+------+----+
+------+------+--------+------+----+



In [ ]:
pandasDF = DIM_BOOKS.toPandas()
pandasDF.to_csv(r'/content/drive/MyDrive/Books_Rating/Fixed_CSV/DIM_BOOKS.csv',mode='w+', encoding='utf-8', index=False)

In [ ]:
spark.stop()